In [1]:
!pip install dfply
import pandas as pd
import numpy as np
from datetime import datetime
import datetime as dt
from dateutil import parser
import re
import math
from dfply import *
from dateutil.relativedelta import *

In [2]:
# trophies_path = "./trophies.csv"
# trophies = pd.read_csv(trophies_path, index_col = 0)

# mval_path = "/content/drive/My Drive/Capstone/Capstone_Data/full_data/markval.csv"
# markval = pd.read_csv(mval_path, index_col = 0)

# players_path = "/content/drive/My Drive/Capstone/Capstone_Data/full_data/Players_Full.csv"
# players = pd.read_csv(players_path, index_col = 0)


def remove_chars(string):
  return string.strip()

def clean_money(string):
  ammount = string
  try:
    if("free" in string.lower()):
      ammount = 0.0
    elif("k" in string):
      ammount = (re.sub('[^\d\.]', '', string))
      ammount = float(ammount) * 1000
    elif("m" in string):
      ammount = (re.sub('[^\d\.]', '', string))
      ammount = float(ammount) * 1000000
  except Exception as e:
    pass
  return ammount

def is_loan(string):
  result = False
  try:
    result = "loan" in string.lower()
  except:
    pass
  return result

def no_loan(string):
  result = False
  try:
    result = "loan" in string.lower()
  except:
    pass
  return not(result)

def clean_loans(string):
  ammount = 0.0
  try:
    if("k" in string):
      ammount = (re.sub('[^\d\.]', '', string))
      ammount = float(ammount) * 1000
    elif("m" in string):
      ammount = (re.sub('[^\d\.]', '', string))
      ammount = float(ammount) * 1000000
  except Exception as e:
    pass
  return ammount

def call_nan(string):
  return math.isnan(string)

def get_season(string):
    season = string
    try:
        season = (re.sub('[^\d\/]', '', string))
    except:
        pass
    return season

def clean_height(string):
    height = string
    try:
        height = (re.sub('[^\d\,]', '', string))
        height = height.replace("," , "")
    except:
        pass
    return height

def clean_position(string):
    position = string
    try:
        position = string.split(",")[0]
        position = position.strip()
    except:
        pass
    return position

def positions_list(string):
    positions = string
    try:
        positions = list(string.split(","))
        positions = list(map(lambda x: x.strip(), positions))
    except:
        pass
    return positions

def check_positions(positions, values):
    result = any(np.in1d(positions, values))
    return result

def filter_by_position(players, position, position_attributes):
    pos_bool = players.player_positions.apply(lambda x: check_positions(x,position))
    position_players = players[pos_bool]
    all_positions = list(map(lambda x: x.lower(), position))
    if (position[0] == 'GK'):
        all_positions = []
    pos_attributes = position_attributes + all_positions
    position_players = position_players[pos_attributes]
    return position_players

def get_position_value(string):
    result = string
    try:
        result = string.split("+")[0].strip()
        result = float(result)
    except Exception as e:
        pass
    return result


In [3]:
def clean_transfers(transfers):
  transfers = transfers.drop_duplicates()
  transfers["Name"] = transfers["Name"].apply(remove_chars)
  transfers["From"] = transfers["From"].apply(remove_chars)
  transfers["To"] = transfers["To"].apply(remove_chars)
  transfers = transfers[transfers["Player_Id"] != "\nAtalanta "]
  transfers.Player_Id = transfers.Player_Id.astype(int)
  transfers.tm_Id = transfers.tm_Id.astype(int)

  loans = transfers[transfers["Fee"].apply(is_loan)]
  transfers = transfers[(transfers["Fee"].apply(no_loan))]
  transfers["Market Value"] = transfers["Market Value"].apply(clean_money)
  transfers["Market Value"] = pd.to_numeric(transfers["Market Value"], errors="coerce")
  transfers["Fee"] = transfers["Fee"].apply(clean_money)
  transfers["Fee"] = pd.to_numeric(transfers["Fee"], errors="coerce")
  transfers["Loan"] = False

  loans["Market Value"] = loans["Market Value"].apply(clean_money)
  loans["Market Value"] = pd.to_numeric(loans["Market Value"], errors="coerce")
  loans["Fee"] = loans["Fee"].apply(clean_money)
  loans["Fee"] = pd.to_numeric(loans["Fee"], errors="coerce")
  loans["Loan"] = True
  loans.fillna(value = 0, inplace=True)

  transfers = transfers.append(loans)
  transfers["Season"] = transfers["Season"].astype('category')

  transfers.columns = ['Player_Id', 'tm_Id', 'Name', 'From', 'To', 'Fee', 'MV',
       'Season', 'Date', 'Loan']

  transfers = (transfers >>
             arrange(X.Player_Id) >>
             mask(X.Fee.notna()) >>
             drop(X.Player_Id))
  transfers.Season = transfers.Season.astype("category")
  transfers["Date"] = pd.to_datetime(transfers["Date"], errors="coerce")

  # hdf_path = "/content/drive/My Drive/Capstone/Capstone_Data/clean_data/data.h5"
  # transfers.to_hdf(hdf_path, key = "transfers", mode='w', format = "table")
  transfers.to_csv("/content/drive/My Drive/Capstone/Capstone_Data/clean_data/Transfers_Clean.csv")
  transfers.to_pickle("/content/drive/My Drive/Capstone/Capstone_Data/clean_data/Transfers_Clean.pkl")


In [4]:
def clean_markval(markval):
  unique_ids = list(players.tm_Id.unique())
  markval = markval[markval.tm_Id.isin(unique_ids)]
  markval.columns = ["Name", "Club", "League", "Season", "MV", "tm_Id"]
  markval = ( markval >>
              mask(X.MV.notna()) >>
              drop(X.Name) 
              )
  markval = ( markval >>
            select(X.tm_Id, X.Club, X.League, X.Season, X.MV))
  markval[markval.tm_Id == 17259]
  markval["Club"] = markval["Club"].apply(remove_chars)
  markval["League"] = markval["League"].apply(remove_chars)
  markval["MV"] = markval["MV"].apply(clean_money)
  markval["MV"] = pd.to_numeric(markval["MV"], errors="coerce")
  markval.Season = markval.Season.astype("category")
  markval.to_csv("/content/drive/My Drive/Capstone/Capstone_Data/clean_data/markval_Clean.csv")
  markval.to_pickle("/content/drive/My Drive/Capstone/Capstone_Data/clean_data/markval_Clean.pkl")


In [5]:
def clean_trophies(trophies):
    trophies.Season = trophies.Season.apply(get_season)
    trophies.Season = trophies.Season.astype("category")
    trophies.sofifa_id = trophies.sofifa_id.astype("int")
    trophies.Name = trophies.Name.apply(remove_chars)
    trophies.Competition = trophies.Competition.apply(remove_chars)
    trophies.Trophy = trophies.Trophy.astype("category")
    trophies.to_csv("trophies_Clean.csv")
    trophies.to_pickle("trophies_Clean.pkl")

In [6]:
def clean_players(players):
    players.Height = players.Height.apply(clean_height)
    players.Height = pd.to_numeric(players.Height, errors = "coerce")
    players.Followers = pd.to_numeric(players.Followers, errors = "coerce")
    players["Date of Birth"] = pd.to_datetime(players["Date of Birth"], errors = "coerce")
    players["Contract Expires"] = pd.to_datetime(players["Contract Expires"], errors = "coerce")
    players.Team = players.Team.apply(remove_chars)
    players.Nationality = players.Nationality.apply(remove_chars)
    players.Name = players.Name.apply(remove_chars)
    players["Strong Foot"] = players["Strong Foot"].astype("category")
    players["Position"] = players["Position"].astype("category")
    players.columns = ['Id', 'tm_Id', 'Name', 'Team', 'Nationality', 'DOB', 'Height',
       'SF', 'Position', 'Joined', 'Contract_Expires', 'Followers']
    players = ( players >>
          arrange(X.Id) >>
          drop(X.Id))
    players.to_csv("Players_Clean.csv")
    players.to_pickle("Players_Clean.pkl")

In [7]:
def clean_sidelined(sidelined):
    datetimeFormat = '%d/%m/%y'
    sidelined = pd.read_csv("sidelined.csv", index_col = 0)
    sidelined.columns = [c.replace(' ', '_') for c in sidelined.columns]
    sidelined['Start_Date'] = pd.to_datetime(sidelined['Start_Date'], format = datetimeFormat)
    sidelined['End_Date'] = pd.to_datetime(sidelined['End_Date'], format = datetimeFormat)
    sidelined['Duration'] = sidelined['End_Date'] - sidelined['Start_Date']
    sidelined['Duration'] = sidelined['Duration'] / np.timedelta64(1, 'D')
    sidelined['Active'] = np.where(sidelined['End_Date'].isna(), True, False)
    sidelined = ( sidelined >>
                  drop(X.Name))
    sidelined.to_csv("sidelined_Clean.csv")
    sidelined.to_pickle("sidelined_Clean.pkl")


In [10]:
def clean_sofifa(players_20, year):
    players_20["main_position"] = players_20.player_positions.apply(clean_position)
    players_20.player_positions = players_20.player_positions.apply(positions_list)
    players_20.main_position = players_20.main_position.astype('category')
    players_20.team_position = players_20.team_position.astype('category')
    players_20.preferred_foot = players_20.team_position.astype('category')
    players_20.joined = pd.to_datetime(players_20.joined)
    players_20.contract_valid_until = pd.to_datetime(players_20.contract_valid_until, format='%Y', errors = 'coerce') 
    players_20.dob = pd.to_datetime(players_20.dob)
    players_20.work_rate = players_20.work_rate.astype('category')
    players_20["game_year"] = year
    players_20.game_year = pd.to_datetime(players_20.game_year, format='%Y', errors = 'coerce')
    players_20.contract_valid_until = players_20.contract_valid_until.apply(add_months)
    all_positions = ["ls","st","rs","lw","lf","cf","rf","rw","lam",
                 "cam","ram","lm","lcm","cm","rcm","rm","lwb",
                 "ldm","cdm","rdm","rwb","lb","lcb","cb","rcb","rb"]
    players_20[all_positions] = players_20[all_positions].apply(lambda x: x.apply(get_position_value) if x.name in all_positions else x)
    return players_20

def get_head_info(players_20):
    players_head = ( players_20 >> 
                   select(X.sofifa_id, X.short_name, X.long_name, X.age, X.dob, X.height_cm, 
                          X.weight_kg, X.nationality, X.club, X.main_position, X.player_positions,
                          X.overall, X.potential, X.preferred_foot, X.weak_foot, X.skill_moves,
                          X.work_rate,X.team_position,X.joined, X.contract_valid_until, X.game_year ))
    return players_head

def add_months(date):
    result = date
    try:
       result = date + relativedelta(months=+5) + dt.timedelta(days=+29)
    except:
        pass
    return result

In [94]:
# players_20 = pd.read_csv("players_20.csv")
# players_20.head(20)


strikers = ['CF', 'ST', 'LS', 'RS']
wingers = ['LW', 'RW', 'LF', 'RF']
clean_midfielders = ['CM', 'LCM', 'RCM','LM', 'RM']
def_midfielders = ['CDM','RDM', 'LDM']
att_midfielders = ['CAM', 'RAM', 'LAM']
full_backs = ['LB', 'RB', 'LWB', 'RWB']
defenders = ['CB', 'LCB', 'RCB']
goalkeepers = ['GK']
striker_attributes = ['sofifa_id', 'player_positions', 'main_position', 'team_position','game_year',
                      'pace', 'shooting', 'passing', 'physic', 'dribbling', 
                      'attacking_crossing','attacking_finishing', 
                      'attacking_heading_accuracy', 'attacking_short_passing',
                    'attacking_volleys', 'movement_acceleration', 
                      'movement_sprint_speed', 'movement_agility',
                      'power_stamina', 'mentality_positioning']
clean_midfielders_attributes = ['sofifa_id', 'player_positions', 'main_position', 'team_position',
                                'game_year',
                                 'passing', 'dribbling', 'attacking_short_passing',
                                'skill_ball_control', 'power_stamina', 'mentality_vision'
                               ]
winger_attributes = ['sofifa_id', 'player_positions', 'main_position', 'team_position','game_year',
                     'pace', 'passing', 'dribbling', 'attacking_crossing', 
                     'attacking_short_passing', 'movement_acceleration','movement_sprint_speed', 
                     'movement_agility', 'power_stamina', 'mentality_vision'] #skill_moves
def_midfielders_attributes = ['sofifa_id',  'player_positions','main_position', 'team_position',
                              'game_year',
                            'defending', 'mentality_aggression', 'mentality_interceptions', 'defending_marking', 
                              'defending_standing_tackle','defending_sliding_tackle']
att_midfielders_attributes = ['sofifa_id', 'player_positions', 'main_position', 'game_year',
                              'team_position','pace', 'passing', 'attacking_short_passing', 'skill_ball_control', 
                              'power_stamina', 'power_long_shots', 'mentality_vision']
defenders_attributes = ['sofifa_id','player_positions','main_position','team_position','game_year',
                        'physic', 'defending', 'mentality_aggression', 
                        'mentality_interceptions','defending_marking', 'defending_standing_tackle',
                        'defending_sliding_tackle', 'attacking_heading_accuracy','power_jumping', 
                        'mentality_composure']
full_backs_attributes = ['sofifa_id','player_positions','main_position','team_position', 'game_year',
                         'pace', 'defending', 'mentality_aggression', 'attacking_crossing',
                        'mentality_interceptions','defending_marking', 'defending_standing_tackle',
                        'defending_sliding_tackle','mentality_composure']
goalkeepers_attributes = ['sofifa_id','player_positions', 'main_position','team_position', 'game_year',
                          'gk_diving','gk_handling','gk_kicking','gk_reflexes',
                          'gk_speed','gk_positioning']
all_positions = ["ls","st","rs","lw","lf","cf","rf","rw","lam",
             "cam","ram","lm","lcm","cm","rcm","rm","lwb",
             "ldm","cdm","rdm","rwb","lb","lcb","cb","rcb","rb"]

# player_attributes = []
# players_20 = clean_sofifa(players_20, year = 2020)
# players_head = get_head_info(players_20)
# players_20

,sofifa_id,player_url,short_name,long_name,age,dob,height_cm,weight_kg,nationality,club,...,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,main_position,game_year
0,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,32,1987-06-24,170,72,Argentina,FC Barcelona,...,66.0,66.0,68.0,63.0,52.0,52.0,52.0,63.0,RW,2020-01-01
1,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,34,1985-02-05,187,83,Portugal,Juventus,...,61.0,61.0,65.0,61.0,53.0,53.0,53.0,61.0,ST,2020-01-01
2,190871,https://sofifa.com/player/190871/neymar-da-sil...,Neymar Jr,Neymar da Silva Santos Junior,27,1992-02-05,175,68,Brazil,Paris Saint-Germain,...,61.0,61.0,66.0,61.0,46.0,46.0,46.0,61.0,LW,2020-01-01
3,200389,https://sofifa.com/player/200389/jan-oblak/20/...,J. Oblak,Jan Oblak,26,1993-01-07,188,87,Slovenia,Atlético Madrid,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GK,2020-01-01
4,183277,https://sofifa.com/player/183277/eden-hazard/2...,E. Hazard,Eden Hazard,28,1991-01-07,175,74,Belgium,Real Madrid,...,63.0,63.0,66.0,61.0,49.0,49.0,49.0,61.0,LW,2020-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18273,245006,https://sofifa.com/player/245006/shuai-shao/20...,Shao Shuai,邵帅,22,1997-03-10,186,79,China PR,Beijing Renhe FC,...,42.0,42.0,43.0,45.0,46.0,46.0,46.0,45.0,CB,2020-01-01
18274,250995,https://sofifa.com/player/250995/mingjie-xiao/...,Xiao Mingjie,Mingjie Xiao,22,1997-01-01,177,66,China PR,Shanghai SIPG FC,...,43.0,43.0,44.0,46.0,47.0,47.0,47.0,46.0,CB,2020-01-01
18275,252332,https://sofifa.com/player/252332/wei-zhang/20/...,Zhang Wei,张威,19,2000-05-16,186,75,China PR,Hebei China Fortune FC,...,49.0,49.0,47.0,47.0,49.0,49.0,49.0,47.0,CM,2020-01-01
18276,251110,https://sofifa.com/player/251110/haijian-wang/...,Wang Haijian,汪海健,18,2000-08-02,185,74,China PR,Shanghai Greenland Shenhua FC,...,48.0,48.0,48.0,48.0,49.0,49.0,49.0,48.0,CM,2020-01-01


In [95]:
def append_or_create(path, df):
    try:
        df1 = pd.read_pickle(path)
        df2 = df1.append(df)
        df2.to_pickle(path)
    except:
        df.to_pickle(path)

def get_all_positions(players_20):
    strikers_20 = filter_by_position(players_20, strikers, striker_attributes)
    wingers_20 = filter_by_position(players_20, wingers, winger_attributes)
    def_mids_20 = filter_by_position(players_20, def_midfielders, def_midfielders_attributes)
    clean_mids_20 = filter_by_position(players_20, clean_midfielders, clean_midfielders_attributes)
    att_mids_20 = filter_by_position(players_20, att_midfielders, att_midfielders_attributes)
    full_backs_20 = filter_by_position(players_20, full_backs, full_backs_attributes)
    defenders_20 = filter_by_position(players_20, defenders, defenders_attributes)
    goalkeepers_20 = filter_by_position(players_20, goalkeepers, goalkeepers_attributes)
    id_vars = ['sofifa_id', 'player_positions', 'main_position',
       'team_position', 'game_year']
    strikers_20_long = pd.melt(strikers_20, id_vars = id_vars, var_name = "Attribute")
    wingers_20_long = pd.melt(wingers_20, id_vars = id_vars, var_name = "Attribute")
    def_mids_20_long = pd.melt(def_mids_20, id_vars = id_vars, var_name = "Attribute")
    clean_mids_20_long = pd.melt(clean_mids_20, id_vars = id_vars, var_name = "Attribute")
    att_mids_20_long = pd.melt(att_mids_20, id_vars = id_vars, var_name = "Attribute")
    full_backs_20_long = pd.melt(full_backs_20, id_vars = id_vars, var_name = "Attribute")
    defenders_20_long = pd.melt(defenders_20, id_vars = id_vars, var_name = "Attribute")
    goalkeepers_long = pd.melt(goalkeepers_20, id_vars = id_vars, var_name = "Attribute")
    
    all_positions_long = pd.concat([strikers_20_long, wingers_20_long, def_mids_20_long, clean_mids_20_long, att_mids_20_long, full_backs_20_long,
                                   defenders_20_long, goalkeepers_long], sort=False)
    
    append_or_create("positions/strikers_Clean.pkl",strikers_20)
    append_or_create("positions/wingers_Clean.pkl",wingers_20)
    append_or_create("positions/def_mids_Clean.pkl",def_mids_20)
    append_or_create("positions/clean_mids_Clean.pkl",clean_mids_20)
    append_or_create("positions/full_backs_Clean.pkl",full_backs_20)
    append_or_create("positions/defenders_Clean.pkl",defenders_20)
    append_or_create("positions/goalkeepers_Clean.pkl",goalkeepers_20)
    append_or_create("sofifa_long_Clean.pkl",all_positions_long)

In [96]:
# get_all_positions(players_20)

In [97]:
# append_or_create("sofifa_head_Clean.pkl", players_head)

,sofifa_id,short_name,long_name,age,dob,height_cm,weight_kg,nationality,club,main_position,...,overall,potential,preferred_foot,weak_foot,skill_moves,work_rate,team_position,joined,contract_valid_until,game_year
0,158023,L. Messi,Lionel Andrés Messi Cuccittini,27,1987-06-24,169,67,Argentina,FC Barcelona,CF,...,93,95,CF,3,4,Medium/Low,CF,2004-07-01,2018-06-30,2015-01-01
1,20801,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,29,1985-02-05,185,80,Portugal,Real Madrid,LW,...,92,92,LW,4,5,High/Low,LW,2009-07-01,2018-06-30,2015-01-01
2,9014,A. Robben,Arjen Robben,30,1984-01-23,180,80,Netherlands,FC Bayern München,RM,...,90,90,SUB,2,4,High/Low,SUB,2009-08-28,2017-06-30,2015-01-01
3,41236,Z. Ibrahimović,Zlatan Ibrahimović,32,1981-10-03,195,95,Sweden,Paris Saint-Germain,ST,...,90,90,ST,4,4,Medium/Low,ST,2012-07-01,2016-06-30,2015-01-01
4,167495,M. Neuer,Manuel Neuer,28,1986-03-27,193,92,Germany,FC Bayern München,GK,...,90,90,GK,4,1,Medium/Medium,GK,2011-07-01,2019-06-30,2015-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18273,245006,Shao Shuai,邵帅,22,1997-03-10,186,79,China PR,Beijing Renhe FC,CB,...,48,56,RES,3,2,Medium/High,RES,2018-07-13,2019-06-30,2020-01-01
18274,250995,Xiao Mingjie,Mingjie Xiao,22,1997-01-01,177,66,China PR,Shanghai SIPG FC,CB,...,48,56,SUB,2,2,Medium/Medium,SUB,2019-02-01,2022-06-30,2020-01-01
18275,252332,Zhang Wei,张威,19,2000-05-16,186,75,China PR,Hebei China Fortune FC,CM,...,48,56,SUB,2,2,Low/Medium,SUB,2019-01-01,2019-06-30,2020-01-01
18276,251110,Wang Haijian,汪海健,18,2000-08-02,185,74,China PR,Shanghai Greenland Shenhua FC,CM,...,48,54,SUB,2,2,Low/Low,SUB,2019-02-01,2022-06-30,2020-01-01


,sofifa_id,player_positions,main_position,team_position,game_year,physic,defending,mentality_aggression,mentality_interceptions,defending_marking,defending_standing_tackle,defending_sliding_tackle,attacking_heading_accuracy,power_jumping,mentality_composure,cb,lcb,rcb
14,155862,[CB],CB,LCB,2015-01-01,82.0,87.0,83,87,85+2,89+3,90+2,86,91,NaN,85.0,85.0,85.0
15,164240,[CB],CB,LCB,2015-01-01,80.0,90.0,76-1,91,90-1,91,89+1,81,90,NaN,85.0,85.0,85.0
21,178603,[CB],CB,SUB,2015-01-01,78.0,88.0,73,90+2,84,89+1,86,91+1,70,NaN,84.0,84.0,84.0
30,139720,[CB],CB,RCB,2015-01-01,81.0,87.0,78,87,85+1,90,85,84,73,NaN,84.0,84.0,84.0
39,177610,"[CB, CDM, CM]",CB,SUB,2015-01-01,85.0,84.0,88,86,79,87+1,88+3,86,70+3,NaN,84.0,84.0,84.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18256,241493,"[CB, RB]",CB,RES,2020-01-01,56.0,47.0,54,47,49,50,44,36,59,38,47.0,47.0,47.0
18262,246694,"[CB, RB]",CB,SUB,2020-01-01,53.0,48.0,46,50,44,52,54,40,70,43,47.0,47.0,47.0
18269,252181,[CB],CB,SUB,2020-01-01,53.0,46.0,41,49,40,48,49,46,75,43,46.0,46.0,46.0
18273,245006,[CB],CB,RES,2020-01-01,51.0,47.0,46,48,45,50,52,40,70,41,46.0,46.0,46.0


,sofifa_id,player_positions,main_position,team_position,game_year,Attribute,value
0,158023,[CF],CF,CF,2015-01-01,pace,93
1,41236,[ST],ST,ST,2015-01-01,pace,76
2,176580,"[ST, CF]",ST,RES,2015-01-01,pace,83
3,7826,[ST],ST,RS,2015-01-01,pace,74
4,167397,[ST],ST,SUB,2015-01-01,pace,77
...,...,...,...,...,...,...,...
12211,251365,[GK],GK,SUB,2020-01-01,gk_positioning,48
12212,252151,[GK],GK,SUB,2020-01-01,gk_positioning,49
12213,245408,[GK],GK,RES,2020-01-01,gk_positioning,48
12214,248182,[GK],GK,RES,2020-01-01,gk_positioning,47
